# Import

Import the 1) AtomSpace, 2) standard AtomSpace types and 3) cheminformatics types

In [ ]:
import mevis as mv
from opencog.atomspace import AtomSpace
from opencog.type_constructors import *
from opencog.cheminformatics import *

## Preliminary definitions

In [ ]:
def assign_bond_color(atom1, atom2):
    if atom1.type_name=='SB' or atom1.type_name=='SB':
        return 'black'
    if atom1.type_name=='DB' or atom1.type_name=='DB':
        return 'blue'
    if atom1.type_name=='TB' or atom1.type_name=='TB':
        return 'cyan'
    if atom1.type_name=='AB' or atom1.type_name=='AB':
        return 'green'
    return 'black'


def assign_bond_size(atom1, atom2):
    if atom1.type_name=='SB' or atom1.type_name=='SB':
        return 5
    if atom1.type_name=='DB' or atom1.type_name=='DB':
        return 10
    if atom1.type_name=='TB' or atom1.type_name=='TB':
        return 15
    if atom1.type_name=='AB' or atom1.type_name=='AB':
        return 7.5
    return 1


def plot_molecule(atomspace, molecule, **kwargs):
    kwargs['graph_height'] = kwargs.get('graph_height', 250)
    fig = mv.plot(
        atomspace,
        filter_target=lambda node: node.type_name != 'Molecule' and contains(molecule, node),
        layout_method='neato',
        node_size=lambda atom: 30 if atom.is_node() else 0,
        node_shape='circle',
        node_label=lambda atom: atom.type_name if atom.is_node() else '',
        node_label_size=30,
        graph_directed=False,
        edge_color=assign_bond_color,
        edge_size=assign_bond_size,
        **kwargs,
    )
    fig.display(inline=True)


def plot_bond(atomspace, bond, **kwargs):
    kwargs['graph_height'] = kwargs.get('graph_height', 100)
    plot_molecule(atomspace, bond, **kwargs)


def contains(atom, target):
    if target == atom:
        return True
    return any(contains(part, target) for part in atom.out)

# Create an AtomSpace

In [ ]:
ats = AtomSpace()
set_default_atomspace(ats)

# Create chemical entities

## 1) Atom

Rules:
- Atoms must have names. The names can be anything.
- Two atoms with the same type and name are treated as identical. Only a single object will be created.

In [ ]:
H('42')

In [ ]:
carbon = C('alpha')
carbon

In [ ]:
O('A particular oxygen atom with a long identifier')

In [ ]:
N('♖ Nitrogen with unicode characters ♘')

In [ ]:
atom1 = H('101')
atom2 = H('102')
atom3 = H('101')  # same hydrogen!

print(atom1 == atom2)
print(atom1 == atom3)

## 2) Bond

Bonds need to contain a list of two atoms.

### Single bond

In [ ]:
bond1 = SB(C('a'), H('b'))
bond1

### Double bond

In [ ]:
bond2 = DB(C('a'), N('b'))
bond2

### Triple bond

In [ ]:
bond3 = TB(C('a'), O('b'))
bond3

### Aromatic bond

In [ ]:
bond4 = AB(C('a'), S('b'))
bond4

In [ ]:
plot_bond(ats, bond1)
plot_bond(ats, bond2)
plot_bond(ats, bond3)
plot_bond(ats, bond4)

## 3) Molecule

- [Methane](https://en.wikipedia.org/wiki/Methane) with four single bonds

In [ ]:
Molecule(
    SB(C('1'), H('1')),
    SB(C('1'), H('2')),
    SB(C('1'), H('3')),
    SB(C('1'), H('4')),
)

- [Carbon dioxide](https://en.wikipedia.org/wiki/Carbon_dioxide) with two double bonds

In [ ]:
carbon_dioxide = Molecule(
    DB(C('a'), O('a')),
    DB(C('a'), O('b')),
)

carbon_dioxide

- [Carbon monoxide](https://en.wikipedia.org/wiki/Carbon_monoxide) with one triple bond

In [ ]:
carbon_monoxide = Molecule(
    TB(C('castor'), O('pollux'))
)

- [Phenol](https://en.wikipedia.org/wiki/Phenol) with 6 aromatic bonds and 7 single bonds

In [ ]:
phenol = Molecule(
    AB(C('3_1'), C('3_2')),
    AB(C('3_2'), C('3_3')),
    AB(C('3_3'), C('3_4')),
    AB(C('3_4'), C('3_5')),
    AB(C('3_5'), C('3_6')),
    AB(C('3_6'), C('3_1')),
    
    SB(C('3_1'), O('3_7')),
    SB(O('3_7'), H('3_8')),
    SB(C('3_2'), H('3_9')),
    SB(C('3_3'), H('3_10')),
    SB(C('3_4'), H('3_11')),
    SB(C('3_5'), H('3_12')),
    SB(C('3_6'), H('3_13')),
)

phenol

## 4) Reaction

In [ ]:
ats = AtomSpace()
set_default_atomspace(ats)

In [ ]:
esterification = BindLink(
    # Variable definition
    VariableList(
        # Typed variables: match to specific atoms
        TypedVariableLink(VariableNode("$carboxylH1"), TypeNode('H')),
        TypedVariableLink(VariableNode("$carboxylO1"), TypeNode('O')),
        TypedVariableLink(VariableNode("$carboxylC1"), TypeNode('C')),
        TypedVariableLink(VariableNode("$carboxylO2"), TypeNode('O')),
        TypedVariableLink(VariableNode("$hydroxylH1"), TypeNode('H')),
        TypedVariableLink(VariableNode("$hydroxylO1"), TypeNode('O')),
        
        # Untyped variables: match to everything
        VariableNode("carboxyl moiety"),
        VariableNode("hydroxyl moiety"),
        GlobNode("rest of carboxyl"),
        GlobNode("rest of hydroxyl"),
    ),
    # Premise: Functional groups found in some educts
    AndLink(
        # 1) Carboxyl group
        Molecule(
            DB(VariableNode("$carboxylC1"), VariableNode("$carboxylO2")),
            SB(VariableNode("$carboxylC1"), VariableNode("$carboxylO1")),
            SB(VariableNode("$carboxylO1"), VariableNode("$carboxylH1")),
            SB(VariableNode("$carboxylC1"), VariableNode("carboxyl moiety")),
            # Globs match one or more bonds by default, but the interval can be changed manually
            GlobNode("rest of carboxyl"),
        ),
        
        # The above will happily match `$carboxyO1` and `carboxy moiety`
        # to the same atom. But we don't want that, so demand that they be distinct.
        NotLink(IdenticalLink(VariableNode("$carboxylO1"), VariableNode("carboxyl moiety"))),
        
        # 2) Hydroxyl group
        Molecule(
            SB(VariableNode("$hydroxylO1"), VariableNode("$hydroxylH1")),
            SB(VariableNode("hydroxyl moiety"), VariableNode("$hydroxylO1")),
            GlobNode("rest of hydroxyl"),
        ),
        NotLink(IdenticalLink(VariableNode("$hydroxylH1"), VariableNode("hydroxyl moiety"))),
        
        # 3) Further constraints
        NotLink(IdenticalLink(
            SB(VariableNode("$hydroxylO1"), VariableNode("$hydroxylH1")),
            SB(VariableNode("$carboxylO1"), VariableNode("$carboxylH1")),
        )),
    ),
    
    # Clause: Formation of products
    AndLink(
        # Produce ester
        Molecule(
            DB(VariableNode("$carboxylC1"), VariableNode("$carboxylO2")),
            SB(VariableNode("$carboxylC1"), VariableNode("$carboxylO1")),
            SB(VariableNode("$carboxylC1"), VariableNode("carboxyl moiety")),
            GlobNode("rest of carboxyl"),
            
            SB(VariableNode("$carboxylO1"), VariableNode("hydroxyl moiety")),
            GlobNode("rest of hydroxyl"),
        ),
        # Produce water
        Molecule(
            SB(VariableNode("$hydroxylO1"), VariableNode("$carboxylH1")),
            SB(VariableNode("$hydroxylO1"), VariableNode("$hydroxylH1")),
        ),
    )
)

In [ ]:
m1 = Molecule(
    DB(C("ca_1"), O("ca_2")),
    SB(C("ca_1"), O("ca_3")),
    SB(O("ca_3"), H("ca_4")),
    SB(C("ca_1"), Fe("ca_5")),
    SB(Fe("ca_5"), Ni("ca_6")),
)

m2 = Molecule(
    SB(O("hy_1"), H("hy_2")),
    SB(C("hy_3"), O("hy_1")),
    SB(C("hy_3"), Zn("hy_4")),
    SB(Zn("hy_4"), Cu("hy_5")),
)

plot_molecule(ats, m1)
plot_molecule(ats, m2)

In [ ]:
from opencog.bindlink import execute_atom

res = execute_atom(ats, esterification)
for i, solution in enumerate(res.out, 1):
    print('# Solution {}'.format(i))
    for j, mol in enumerate(solution.out, 1):
        print('## Molecule {}'.format(j))
        plot_molecule(ats, mol)